In [66]:
import numpy as np
import matplotlib.pyplot as plt
import epics as ep
import h5py as h5
from time import sleep

## User inputs

In [128]:
cameraString = 'SATOP11-PSCR090:'
AdjustName = 'SATFE10-OAPU040:'
SaveFN = cameraString[:-1]+'6.h5'
numshots =5

## grab image and plot

In [129]:
def getImg(cameraString,AdjustName,numshots):
    imagePV =cameraString+'FPICTURE'
    widthPV =cameraString+'WIDTH'
    heightPV =cameraString+'HEIGHT'
    screenPOSPV =cameraString+'MOTOR_PROBE.RBV'
    
    if AdjustName == 'SATFE10-OAPU040:':
        Adjust_XPV = AdjustName+'MOTOR_X.VAL'
        Adjust_YPV = AdjustName+'MOTOR_Y.VAL'
        Adjust_WPV = AdjustName+'MOTOR_W.VAL'
        Adjust_HPV = AdjustName+'MOTOR_H.VAL'
        
        Adjust_X = ep.caget(Adjust_XPV)
        Adjust_Y = ep.caget(Adjust_YPV)
        Adjust_W = ep.caget(Adjust_WPV)
        Adjust_H = ep.caget(Adjust_HPV)        

    width = int(ep.caget(widthPV))
    height = int(ep.caget(heightPV))
    screenPOS = ep.caget(screenPOSPV)
    tmp_Img = []
    for i in range(0,numshots):
        tmp_Img.append(ep.caget(imagePV)[0:height*width].reshape([height, width]))
        sleep(0.3)
    tmp_Img = np.asarray(tmp_Img)
    Img = {
        "images": tmp_Img,
        "mean": tmp_Img.mean(axis=0),
        "Camera name": cameraString,
        "Screen position": screenPOS,
        "Width": width,
        "Height": height,
        "AdjustName":AdjustName,
        "Adjust_X":Adjust_X,
        "Adjust_Y":Adjust_X,
        "Adjust_W":Adjust_W,
        "Adjust_H":Adjust_H
    }
    return Img

In [130]:
Img = getImg(cameraString,AdjustName,numshots)

In [131]:
with h5.File('run02/'+SaveFN,'w-') as fh:
    for k,v in Img.items():
        fh[k] = v